In [2]:
from custom_env import CustomEnv
from config import Config

cfg = Config()
env = CustomEnv(cfg)
env.reset()

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoSarcArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {ne

MyoSuite:> Registering Myo Envs
    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


(array([ 2.0205e+00, -6.6971e-04,  9.5000e-01, -4.2050e-01,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -9.7163e-02,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  1.3680e-01,  1.4842e-01,  7.0695e-01,
        -7.4050e-01, -3.4323e-01,  8.0550e-01, -1.0284e-01, -8.2880e-02,
        -7.3042e-01,  6.3200e-02,  7.0000e-01,  7.5030e-02, -2.9673e-01,
         7.2266e-01, -1.3614e-01,  2.6707e-01,  3.5348e-01,  6.5982e-01,
        -1.5184e-01,  4.2417e-01,  5.1843e-01,  9.1904e-01, -2.0944e-01,
         2.5921e-01,  5.1057e-01,  7.9335e-01, -2.0420e-01,  2.2780e-01,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0

In [3]:
obs, reward, terminated, truncated, info = env.step(env.action_space.sample())  # take a random action

In [5]:
info['obs_dict'].keys()

dict_keys(['time', 'pelvis_pos', 'body_qpos', 'body_qvel', 'ball_pos', 'ball_vel', 'paddle_pos', 'paddle_vel', 'paddle_ori', 'padde_ori_err', 'reach_err', 'palm_pos', 'palm_err', 'touching_info', 'act'])

In [4]:
info['obs_dict']['touching_info']

array([0, 0, 0, 0, 0, 0])

In [5]:
info['obs_dict']['reach_err']

array([ 2.713 ,  0.7649, -0.2112])

In [ ]:
from metrics.auc import compute_auc, load_tb_scalar, smooth_values

In [4]:
logdir = "./logs/exp8/RecurrentPPO_1"
tag = "rollout/ep_rew_mean"

steps, values = load_tb_scalar(logdir, tag)

# Apply smoothing
values_smooth = smooth_values(values, window=10)

# Compute AUC
auc = compute_auc(steps, values_smooth)

print(f"AUC for tag '{tag}': {auc:.2f}")

AUC for tag 'rollout/ep_rew_mean': 3094297182.41
